![CC-BY-SA](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by-sa.svg)


This notebook was created by [Bernardo Freitas Paulo da Costa](http://www.im.ufrj.br/bernardofpc),
and is licensed under Creative Commons BY-SA.

Antes de enviar este Teste, verifique que tudo está funcionando como esperado.
Por exemplo, **rode o código inteiro, do zero**.
Para isso, vá no menu, escolha _Kernel_, depois _Restart & Run All_.

Verifique, também, que você respondeu todas as questões:
* as questões de código têm `YOUR CODE HERE` (e você pode apagar o `raise NotImplemented` ao incluir sua resposta)
* as questões discursivas têm "YOUR ANSWER HERE".

---

In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Convergência de vetores na iteração QR

Vamos ver como os vetores das matrizes $Q$ do algoritmo QR iterativo convergem para os autovetores de $A = QDQ^T$.

## Gerando matrizes diagonalizáveis aleatórias

Para inclusive poder verificar sem problemas a qualidade da diagonalização,
vamos gerar matrizes ortogonais aleatórias $Q$ e matrizes diagonais aleatórias $D$.

Uma forma muito simples de gerar matrizes ortogonais aleatórias é fazer a fatoração QR de uma matriz $A$ qualquer
(veja a função `np.linalg.qr`).

### Questão 1. Parte ortogonal

Escreva uma função que, dado $n$, retorne uma matriz aleatória $n \times n$ ortogonal.

In [3]:
def randq(n):
    # YOUR CODE HERE
    A = np.random.rand(n, n)   # Cria uma matriz aleatória
    Q = np.linalg.qr(A)[0]  # Pega a parte ortogonal da decomposição QR
    
    return Q

Verificando que ela é ortogonal:

In [4]:
i = np.identity(20)
for _ in range(10):
    Q = randq(20)
    assert np.allclose(Q.T @ Q, i)

Verificando que é aleatória

In [5]:
np.random.seed(1)
for _ in range(10):
    n = np.random.randint(30,100)
    prev = randq(n)
    for _ in range(3):
        nova = randq(n)
        assert np.linalg.norm(prev - nova)**2 >= n/2

### Questão 2. Matriz diagonalizável

Agora, escreva uma função que, dado $n$, retorna uma matriz diagonalizável aleatória $n \times n$,
cujos autovalores sejam reais aleatórios entre $-n$ e $n$.
Retorne também os autovalores num vetor `a_val`
e os autovetores correspondentes numa lista (de arrays `numpy`) `a_vet`.

In [27]:
import random


def rand_diag(n):
    """Creio que seja interessante definirmos aleatoriamente os autovalores e os autovetores, e assim sim achar a matriz usando a decomposição espectral."""
    
    a_val = [random.uniform(-n, n) for _ in range(n)]
    D = np.diag(a_val)

    P = randq(n)    # Criando uma matriz ortogonal nxn que usaremos como base de autovetores.
    
    A = np.matmul(np.matmul(P,D), P.T)
    
    return A, np.array(a_val), P.T

Verificando que realmente vale $Av = \lambda v$:

In [23]:
for _ in range(2):
    A,ls,vs = rand_diag(3)
    for l,v in zip(ls,vs):
        print(A@v - l*v)

[ 0.00000000e+00 -5.55111512e-17  0.00000000e+00]
[-2.22044605e-16  0.00000000e+00  0.00000000e+00]
[ 0.00000000e+00  0.00000000e+00 -1.11022302e-16]
[ 2.22044605e-16 -2.77555756e-17  0.00000000e+00]
[ 3.33066907e-16 -1.33226763e-15 -5.55111512e-17]
[-2.22044605e-16 -1.66533454e-16  2.22044605e-16]


In [25]:
for _ in range(20):
    A,ls,vs = rand_diag(30)
    for l,v in zip(ls,vs):
        assert np.allclose(A@v, l*v)

In [28]:
np.random.seed(2)
for _ in range(20):
    A,ls,vs = rand_diag(35)
    # Há (bastantes) autovalores positivos e negativos
    assert sum(ls < 0) > 5
    assert sum(ls > 0) > 5

### Questão 3. Autovetores ordenados

Modifique rand_diag para que os autovalores venham em ordem decrescente do módulo.
A função `argsort` pode ser útil: não queremos mudar o sinal dos autovalores, mas precisamos ordenar `abs(a_val)`.

In [39]:
def rand_diag(n):
    # YOUR CODE HERE
    
    a_val = np.array([random.uniform(-n, n) for _ in range(n)])

    absoluto = abs(a_val)
    ordem = reversed(np.argsort(absoluto))

    a_val_sorted = []
    for indice in ordem:
        a_val_sorted.append(a_val[indice])
    a_val_sorted = np.array(a_val_sorted)

    D = np.diag(a_val_sorted)

    P = randq(n)
    
    A = np.matmul(np.matmul(P,D), P.T)
    
    return A, a_val_sorted, P.T

In [40]:
np.random.seed(2)
for _ in range(20):
    A,ls,vs = rand_diag(30)
    for l1,l2 in zip(ls[:-1], ls[1:]):
        assert abs(l1) > abs(l2)
    for l,v in zip(ls,vs):
        assert np.allclose(A@v, l*v)
    # Há (bastantes) autovalores positivos e negativos
    assert sum(ls < 0) > 5
    assert sum(ls > 0) > 5

## CVGA Projetivo

Vamos usar o ângulo entre vetores para obter uma velocidade de convergência.
Lembre que $\cos \theta = \frac{u\cdot v}{|u| \cdot |v|}$.

Se temos uma base ortogonal $e_i$, e uma base $u_i = \pm e_i$,
elas estão "alinhadas", por mais que os vetores possam estar com o sentido trocado.
Por isso, não vamos usar o ângulo ("orientado") dado pela fórmula acima,
mas o **menor** ângulo $\hat\theta$ entre as retas determinadas por $u$ e $v$.
Este é bastante parecido:
$$ \cos \hat\theta = \frac{|u\cdot v|}{|u| \cdot |v|}. $$

Este ângulo é importante em geometria projetiva, porque apenas a direção das retas importa, não a sua orientação.

### Questão 4. Função ângulo

Usando a fórmula anterior, escreva uma função que calcula o ângulo entre as retas de direção $u$ e $v$.

In [43]:
from numpy.linalg import norm as norma

def proj_angle(u,v):
    u = np.array(u)
    v = np.array(v)
    
    numerador = norma(u@v)
    denominador = norma(u)*norma(v)

    angulo = np.arccos(numerador/denominador)

    return angulo

In [44]:
assert(proj_angle([1,0], [0,1]) == np.pi/2)
assert(proj_angle([1,0], [1,0]) == 0)

## Algoritmo QR: iteração de bases

O algoritmo QR iterado, dada uma matriz $A$,
repetidamente usa a fatoração QR para diagonalizá-la.
Começando com uma matriz $Q_0$ qualquer (em geral, a identidade), em cada etapa, o algoritmo

1. Fatora a matriz $A Q_{n-1}$ como $Q_n R_n$
2. Se $R_n$ for "quase diagonal" (use a função `np.allclose`), retorna
3. Senão, itera

Estas operações correspondem a aplicar a ideia da iteração direta não apenas em um vetor, mas em uma _base_.

### Questão 5. QR com bases

Escreva a função `itera_qr` que retorna as matrizes $Q_n$ e $R_n$ em listas.
A matriz $Q_n$ pode ser vista como tendo em suas colunas uma "base" de vetores
que está "convergindo" para a base de autovetores de $A$.

In [94]:
def itera_qr(A, maxiter=2000):
    Q0 = np.identity(A.shape[0])
    Qs = []
    Rs = []
    # YOUR CODE HERE
    for _ in range(maxiter):
        Q1, R1 = np.linalg.qr(np.matmul(A,Q0))
        Qs.append(Q1)
        Rs.append(R1)

        R1_diag = np.diag(R1)   # Extraio a diagonal de R1
        Diag_matrix = np.diag(R1_diag)  # Crio uma matrix, com valores apenas na diagonal.

        if np.allclose(R1, Diag_matrix, rtol=1e-5, atol=1e-5):
            return Qs, Rs
        
        Q0 = Q1
    raise ValueError('Não convergiu')

#### 5.1 Qn ortogonais

Este é fácil.

In [98]:
np.random.seed(3)
for n in np.random.randint(4,8, size=10):
    A, _, _ = rand_diag(n)
    i = np.identity(n)
    Qs,_ = itera_qr(A)
    for Qn in Qs:
        assert np.allclose(i, Qn @ Qn.T)

ValueError: Não convergiu

#### 5.2 $R_n$ quase diagonal.

No fim do algoritmo, teremos em $R_n$ a candidata para a matriz dos autovalores de $A$.

(Observação: dependendo da versão do numpy que você usar, pode ser que a diagonal de $R$ tenha apenas valores positivos.  Neste caso, insira um comentário abaixo, e troque o segundo `assert` para `abs(a_vals)`, para verificar no seu computador se o seu algoritmo está funcionando.)

In [108]:
np.random.seed(32)
for n in np.random.randint(4,8, size=10):
    A, a_vals, _ = rand_diag(n)
    _,Rs = itera_qr(A, maxiter=5000)
    d_Rs = Rs[-1].diagonal()
    assert np.allclose(Rs[-1], np.diag(d_Rs))
    assert np.allclose(d_Rs, a_vals)

ValueError: Não convergiu

### 6. Distância entre bases

Escreva uma função que, dadas duas matrizes $B_1$ e $B_2$ (representando bases do $R^n$),
calcula as distâncias entre os vetores **linhas** correspondentes, e as retona num array ou lista.
(usaremos vetores linhas, e não colunas, porque são estes que vêm na lista `a_vec`;
atenção, a matriz $Q$ tem a base nas colunas...)

In [ ]:
def base_distances(B1,B2):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
I3 = np.identity(3)
assert all(base_distances(I3, I3) == np.array([0,0,0]))
assert all(base_distances(I3, -I3) == np.array([2,2,2]))

In [ ]:
I3 = np.identity(3)
assert all(base_distances(I3, [[0,1,0], [0,0,1],[1,0,0]]) == np.sqrt(np.array([2,2,2])))
assert all(base_distances(I3, [[1,0,0], [0,1,0], [1,0,1]]) == np.array([0,0,1]))

### 7. Autovetores

Com ajuda função anterior, faça um gráfico da evolução da distância (euclidiana)
entre os autovetores de uma matriz $3\times 3$ (aleatória, diagonalizável)
e os vetores correspondentes vindos da iteração QR.

In [ ]:
np.random.seed(5)
A, _, a_vec = rand_diag(3)
Qs,_ = itera_qr(A)
# YOUR CODE HERE
raise NotImplementedError()

### 8. Ângulos

Agora, em vez de calcular a distância entre os vetores,
faça o gráfico do ângulo entre as retas respectivas,
como já fizemos no início.

Aproveite a ideia da função que calcula a distância entre bases para fazer uma `base_angles` análoga.

In [ ]:
def base_angles(B1,B2):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
np.random.seed(6)
A, _, a_vec = rand_diag(4)
Qs,Rs = itera_qr(A)
# YOUR CODE HERE
raise NotImplementedError()
plt.legend(['$v_{}$'.format(i) for i in range(4)], loc=0);

### 9. Velocidade de convergência

Use os ângulos calculados para estimar a taxa de convergência de cada vetor para o autovetor correspondente.
Como isso se relaciona com os autovalores calculados?

In [ ]:
### Contas aqui
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

### 10. Precisão para o ângulo projetivo

Se você fez corretamente o gráfico anterior,
algumas linhas pararam de aparecer por volta de $10^{-8} = \sqrt{\epsilon}$.
Isso se deve à imprecisão do computador ao calcular $\arccos(1 - x)$ com $x$ muito pequeno.
Para contornar isso, vamos usar uma fórmula estável para o ângulo.

Se temos dois vetores $u$ e $v$, podemos projetar (como fizemos) $v$ em $u$ e obter o cosseno do ângulo entre eles.
Para contornar o problema de precisão quando $u$ e $v$ estão quase alinhados,
vamos dar mais informação ao nosso programa: vamos calcular o **seno** do ângulo também.
Para isso, "Gram-Schmidt": basta retirar de $v$ a componente na direção de $u$.
O que restar será ortogonal a $u$, e será proporcional ao seno do ângulo entre $v$ e $u$.
Faça um desenho para ver!

Com todos os detalhes: dados $u$ e $v$,
determine a projeção de $v$ em $u$ (com um produto interno) e o vetor restante por subtração.
Agora, use a função `arctan2` que recebe dois comprimentos $y$ e $x$ (nesta ordem "errada")
e dá o ângulo do ponto $(x,y)$ com relação ao eixo X.
Como esta função usa ambas as coordenadas, ela não sofre os mesmos problemas de precisão que `arccos`.

In [ ]:
def proj_angle_prec(u,v):
    # YOUR CODE HERE
    raise NotImplementedError()

Testando ângulos "normais"

In [ ]:
np.random.seed(98765)
for n in np.random.randint(4,10,size=10):
    u = np.random.rand(n)
    v = np.random.rand(n)
    assert proj_angle(u,v) - proj_angle_prec(u,v) < 1e-15

Testando ângulos "pequenos"

In [ ]:
assert proj_angle_prec([1,0,0,0], [1,0,1e-15,1e-15]) == np.sqrt(2)*1e-15 - 2e-31

### 11. Convergência das bases

Faça agora o gráfico do ângulo entre as bases da iteração QR,
e veja-os diminuirem até $10^{-16}$ como esperado.

In [ ]:
np.random.seed(999)
A, _, a_vec = rand_diag(5)
Qs,_ = itera_qr(A)
# YOUR CODE HERE
raise NotImplementedError()